In [1]:
!pip install datasets
from datasets import Dataset, DatasetDict
import pandas as pd
from huggingface_hub import notebook_login

In [ ]:
# notebook_login()

In [2]:
!pip install transformers
from datasets import load_dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("davidgaofc/TechDebtClassifier")
techdebt = load_dataset("davidgaofc/techdebt", streaming=True)

In [3]:
def preprocess_function(examples):
    return tokenizer(examples["Diff"], padding='max_length', truncation=True)

tokenized_techdebt = techdebt.map(preprocess_function, batched=True)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def format_dataset(examples):
    examples['label'] = examples['FaultInducingLabel']
    return examples

tokenized_techdebt = tokenized_techdebt.map(format_dataset)
tokenized_techdebt

{'train': <datasets.iterable_dataset.IterableDataset at 0x790f412cb370>,
 'test': <datasets.iterable_dataset.IterableDataset at 0x790f412cb520>,
 'validation': <datasets.iterable_dataset.IterableDataset at 0x790f412cb850>}

In [4]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("davidgaofc/TechDebtClassifier", num_labels=2)

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/334M [00:00<?, ?B/s]

In [4]:
!pip install accelerate -U
!pip install transformers[torch]
!pip install torch -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [9]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_eval_batch_size=16)
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
f1_score = load_metric("f1")
precision_met = load_metric("precision")
recall_met = load_metric("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    f1 = f1_score.compute(predictions=predictions, references=labels, average="weighted")
    precision = precision_met.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_met.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"], "precision": precision['precision'], "recall": recall['recall']}


trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=tokenized_techdebt['train'],
    eval_dataset=tokenized_techdebt['test'],
    compute_metrics=compute_metrics,
)

In [10]:
trainer.evaluate()

{'eval_loss': 0.13908785581588745,
 'eval_accuracy': 0.9541031017280023,
 'eval_f1': 0.9492902966412665,
 'eval_precision': 0.9494100297203545,
 'eval_recall': 0.9541031017280023,
 'eval_runtime': 1080.5708,
 'eval_samples_per_second': 63.999,
 'eval_steps_per_second': 4.001}

In [ ]:
!pip show accelerate

Name: accelerate
Version: 0.24.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, torch
Required-by: 


In [ ]:
# training_args = TrainingArguments(output_dir="training", learning_rate=2e-5, weight_decay=0.01,
#     num_train_epochs=1,
#     per_device_train_batch_size=30,
#     per_device_eval_batch_size=30,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_techdebt["train"],
#     eval_dataset=tokenized_techdebt["validation"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
# )

In [ ]:
# trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
# trainer.evaluate()

In [ ]:
# model.save_pretrained("techdebt_model"+"1")
# tokenizer.save_pretrained("techdebt_tokenizer"+"1")

In [ ]:
# !huggingface-cli login
# trainer.push_to_hub()



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

model.safetensors:   0%|          | 0.00/334M [00:00<?, ?B/s]

events.out.tfevents.1699762438.cb8aefb507c7.1858.0:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

Upload 9 LFS files:   0%|          | 0/9 [00:00<?, ?it/s]

events.out.tfevents.1699762464.cb8aefb507c7.1858.1:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

...

events.out.tfevents.1699763173.cb8aefb507c7.1858.4:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

events.out.tfevents.1699763185.cb8aefb507c7.1858.5:   0%|          | 0.00/6.69k [00:00<?, ?B/s]

events.out.tfevents.1699772699.cb8aefb507c7.1858.6:   0%|          | 0.00/359 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

'https://huggingface.co/davidgaofc/training/tree/main/'

In [ ]:
import numpy as np
!pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00


In [ ]:
model_checkpoint = "davidgaofc/TechDebtClassifier"  # Replace with your checkpoint directory
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
training_args = TrainingArguments(output_dir="training2", learning_rate=2e-5, weight_decay=0.01, num_train_epochs=1, per_device_train_batch_size=30, per_device_eval_batch_size=30,push_to_hub=True,)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_techdebt["train"],
    eval_dataset=tokenized_techdebt["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

NameError: ignored

In [ ]:
trainer.train()

Step,Training Loss
500,0.089000
1000,0.112600
1500,0.110900
2000,0.103300
2500,0.094100
3000,0.103700
3500,0.108100
4000,0.108700
4500,0.114700
5000,0.108200


Step,Training Loss
500,0.089000
1000,0.112600
1500,0.110900
2000,0.103300
2500,0.094100
3000,0.103700
3500,0.108100
4000,0.108700
4500,0.114700
5000,0.108200


In [ ]:
trainer.evaluate()